# BACKTESTING VALIDACION

## TEORIA

### LOGICA (IDEA)

Sistema de Larry Connors RSI2

* Activo:
  * SPX

* Long:
  * Entrada
    * Rsi2 < 10 & Precio > Sma 200
  * Salida
    * Rsi2 > 50 | Precio < Sma 200

### SISTEMA

* Entrada -> data IS
* Evitar sesgo de anticipacion
* Nunca mezclar long y short en el mismo sistema -> mejor dos sistemas
* Buscamos:
  * Sistemas que funcionen bien
  * Sistemas que funcionen mal
* Salida -> Señales y Position Size

* Es el caso mas complejo (optimizado):
  * Entrada y salida por indicador
  * Y que no este dentro de la posicion mas de un numero de velas
  * Y que salga por take profit o stop loss
* Caso mas simple:
  * Entrada por indicador
  * Salida por ...
  * Habria que programarlo...
* CONTROL ABSOLUTO DE LAS SEÑALES:
  * Tiempo
  * Pnl
* Solo Importa:
  * data.TRADE
  * data.ROID
  * data.ROIACUM

### BACKTESTING RESULTADO

* Entrada
  * Señales y Position Size
  * Resultados del sistema

### BACKTESTING OPTIMIZACION

* Encontrar los parametros que hacen mas robusto al sistema
* Sistemas con poco parametros son los mejores
* Si el sistema tiene mucho parámetros lo podemos torturar hasta que nos diga lo que queremos
* La data debe comenzar en el mismo punto, el calculo de indicadores puede afectar
* Nuestro sistema:
  * Sma -> 200
  * Rsi -> 2, 10, 50

## IMPORTACIONES

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 

import matplotlib.pyplot as plt

from IPython.display import HTML, display

import warnings
warnings.filterwarnings('ignore')

## DATA IS

In [ ]:
data = pd.read_excel('dfIS.xlsx', index_col=0)
data

## SETUP

In [ ]:
perSma = 200
perRsi = 2

rsiIn  = 10
rsiOut = 50

salidaVelas = 0

sentido = 'long'

pSize = 1

tp = 0
sl = 0

# se aplica en la entrada para la entrada y la salida
# 0.01 es 0.01% se aplica a la entrada; $10000 -> $1
# 0.02 es 0.02% se aplica a la entrada; $10000 -> $2
comision = 0 
slippage = 0 

## SISTEMA

In [ ]:
def ocpSma(df, periodo = 20, borraNan = False, col = 'Close'):
    
    ''' return data'''
    
    df[f's{periodo}'] = df[col].rolling(periodo).mean()
    
    if borraNan: df.dropna(inplace = True)
        
    return df
    
def ocpRsi(df, periodo = 14, borraNan = False, col = 'Close'):
    
    ''' return data'''
    
    df['dif'] = df[col].diff()

    df['win']     = np.where(df['dif'] > 0, df['dif'], 0)
    df['loss']    = np.where(df['dif'] < 0, abs(df['dif']), 0)
    df['emaWin']  = df.win.ewm(span = periodo).mean()
    df['emaLoss'] = df.loss.ewm(span = periodo).mean()
    df['rs']      = df.emaWin / df.emaLoss

    df[f'rsi{periodo}'] = 100 - (100 / (1+df.rs))

    df.drop(['dif', 'win', 'loss', 'emaWin', 'emaLoss', 'rs'], axis = 1, inplace = True)

    if borraNan: df.dropna(inplace = True)

    return df



# P  -> Posicion
# cP -> cierre Posicion

def dameSistema(df, perSma, perRsi, rsiIn, rsiOut):
    
    df['signal'] = ''
    
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    
    #df['signal'] = np.where((df[f'rsi{perRsi}'] < rsiIn) , 'P' , df.signal)
    #df['signal'] = np.where((df[f'rsi{perRsi}'] > rsiOut), 'cP', df.signal)
    #data['signal'] = np.where(data.signal.shift(5) == 'P' ,'cP', df.signal)

    df['signal'] = np.where((df[f'rsi{perRsi}'] < rsiIn) & (df.Close > df[f's{perSma}']) , 'P' , df.signal)
    #df['signal'] = np.where((df[f'rsi{perRsi}'] > rsiOut) , 'cP', data.signal)
    df['signal'] = np.where((df[f'rsi{perRsi}'] > rsiOut) | (df.Close < df[f's{perSma}']) , 'cP', df.signal)
    
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>  
    # evitar sesgo de anticipacion
    df['position'] = ''
    df['position'] = df.signal.shift()

    return df

def damePosition(df):

    df['tradeInd'] = ''
    
    inTrade = False
    for i in df.index:
        
        cell = df.loc[i, 'position']
    
        if pd.isna(cell):
            pass # Sin señal, no cambiamos estado
        else:
            if cell == 'P':
                if not inTrade:
                    inTrade = True
                    df.loc[i, 'tradeInd'] = 'In'  # Entrada
                else: # Ya en posicion
                    df.loc[i, 'tradeInd'] = 'p'
            elif cell == 'cP':
                if inTrade:
                    df.loc[i, 'tradeInd'] = 'Out'  # Salida
                inTrade = False
            else:
                if inTrade:
                    df.loc[i, 'tradeInd'] = 'p'
    
    df['TRADE'] = df.tradeInd

    return df


def dameSalidaVelas(df, num):

    '''
    Si num vale 0 no se tiene en cuenta la salida por velas
    '''

    df['velas'] = 0
    df = df.reset_index(drop=False)
    for i in df.index:
    
        if i > 1:
        
            cell = df.loc[i, 'TRADE']
            
            if   cell == 'In' : df.loc[i, 'velas'] = 1
            elif cell == 'p'  : df.loc[i, 'velas'] = df.loc[i-1, 'velas'] +1
                
    numMaxVelas = num
    if numMaxVelas:
        
        df['tradeVela'] = ''
        
        
        for i in df.index:
        
            cell = df.loc[i, 'TRADE']
            vela = df.loc[i, 'velas']
            
            if vela <= numMaxVelas: df.loc[i, 'tradeVela'] = cell if cell in ['In', 'p'] else ''
        
        df['tradeVela'] = np.where((df['tradeVela'] == '') & (df['tradeVela'].shift().isin(['In', 'p'])),'Out',df['tradeVela'])
        
        df['TRADE'] = df.tradeVela
    
    else:
    
        df['tradeVela'] = df.TRADE

    df.set_index('Date', inplace=True)
    
    return df
    
    
def dameSalidaPnlOLD(df, sentido, tp, sl):

    '''
    tp valores tipo 3.5
    sl valores tipo -1.5

    si alguno vale 0 no se tiene en cuenta ese parametro
    '''

    df['ROID'] = 0
    df = df.reset_index(drop=False)
    for i in df.index:
        
        estado = df.loc[i, 'TRADE']
       
        if i > 0:
            # sentido long
            if   estado == 'In'   : df.loc[i, 'ROID'] = (df.loc[i, 'Close'] / df.loc[i, 'Open'] -1) * 100
            elif estado == 'p'    : df.loc[i, 'ROID'] = (df.loc[i, 'Close'] / df.loc[i-1, 'Close'] - 1) * 100
            elif estado == 'Out'  : df.loc[i, 'ROID'] = (df.loc[i, 'Open']  / df.loc[i-1, 'Close']  - 1) * 100
            else                  : df.loc[i, 'ROID'] = 0
    
    if sentido == 'short': df['ROID'] = - df.ROID

    df['ROIACUM'] = 0
    for i in df.index:
        estado = df.loc[i, 'TRADE']
        roiD   = df.loc[i, 'ROID']
    
        if estado == 'In'          : df.loc[i, 'ROIACUM'] = roiD
        elif estado in ['p', 'Out']: df.loc[i, 'ROIACUM'] = ((1+roiD/100) * (1+df.loc[i-1, 'ROIACUM']/100) -1)*100
    

    df['tradePnl'] = ''
    for i in df.index:
    
        cell    = df.loc[i, 'TRADE']
        roiAcum = df.loc[i, 'ROIACUM']
        
        if   cell == 'In': df.loc[i, 'tradePnl'] = 'In'
        elif cell == 'p':
            #if ((tp != 0 and df.loc[i-1, 'ROIACUM'] > tp) or (sl != 0 and df.loc[i-1, 'ROIACUM'] < sl)): df.loc[i, 'tradePnl'] = 'Out'
            if ((tp != 0 and df.loc[i, 'ROIACUM'] > tp) or (sl != 0 and df.loc[i, 'ROIACUM'] < sl)): df.loc[i, 'tradePnl'] = 'Out'
            else                                                                                       : df.loc[i, 'tradePnl'] = 'p'
        elif cell == 'Out': df.loc[i, 'tradePnl'] = 'Out'

    df['tradePnl'] = np.where((df['tradePnl'] == 'p')   & (df['tradePnl'].shift().isin(['', 'Out'])),'',df['tradePnl'])
    df['tradePnl'] = np.where((df['tradePnl'] == 'Out') & (df['tradePnl'].shift().isin(['', 'Out'])),'',df['tradePnl'])

    df['TRADE'] = df.tradePnl
    
    df.set_index('Date', inplace=True)
    
    return df


def dameSalidaPnl(df, sentido, tp, sl, comision, slippage):

    '''
    tp valores tipo 3.5
    sl valores tipo -1.5
    
    si alguno vale 0 no se tiene en cuenta ese parametro
    
    se aplica en la entrada para la entrada y la salida
    comision 0.01 es 0.01% se aplica a la entrada; $10000 -> $1
    slippage 0.02 es 0.02% se aplica a la entrada; $10000 -> $2

    '''

    def pnlSalida(Open, High, Low, Close, precioRef, sentido, tp, sl):
        '''
        Determina si se alcanza SL/TP o es sesion normal durante una vela
    
        PARAMETROS:
        - Open, High, Low, Close: Precios OHLC de la vela
        - precioRef: Precio de referencia (entrada de la posicion)
        - sentido: 'long' o 'short'
        - sl: Stop Loss en % (ej: -2.0 para -2%), si 0 no se considera
        - tp: Take Profit en % (ej: 0.8 para +0.8%), si 0 no se considera
       
        LOGICA:
        - Prioridad: SL primero, luego TP. En una sesion se mira primero el SL, y luego el TP
        - Si se supera en apertura, ROI real de apertura
        - Si se supera durante vela, ROI teorico por SL/TP
    
        RETORNA:
        - Tupla (tipoSalida, roiPorcentaje)
        - tipoSalida: sltp, o normal
        - roiPorcentaje: ROI REAL en % respecto a precioRef
        '''

        # Funcion auxiliar para calcular ROI real
        def calcularRoi(precio, precioRef, sentido):
            
            roi = (precio - precioRef) / precioRef * 100
            if sentido == 'short': roi = -roi
                
            return roi
    
        # VERIFICAR SL (PRIORIDAD 1)
        if sl:
            
            if sentido == 'long':
                
                precioSl = precioRef * (1 + sl/100)
                
                # Verificar si se toca SL
                if Open <= precioSl or Low <= precioSl:
                    # Si se toca en apertura, usar ROI de apertura
                    if Open <= precioSl: return ('sltp', calcularRoi(Open, precioRef, sentido))
                    # Se toca durante la vela, usar SL teorico    
                    else: return ('sltp', sl)
            
            # short           
            else:  
                
                precioSl = precioRef * (1 - sl/100)
                
                # Verificar si se toca SL
                if Open >= precioSl or High >= precioSl:
                    # Si se toca en apertura, usar ROI de apertura
                    if Open >= precioSl: return ('sltp', calcularRoi(Open, precioRef, sentido))
                    # Se toca durante la vela, usar SL teorico   
                    else: return ('sltp', sl)
    
        # VERIFICAR TP (PRIORIDAD 2)
        if tp:
            
            if sentido == 'long':
                
                precioTp = precioRef * (1 + tp/100)
                
                # Verificar si se toca TP
                if Open >= precioTp or High >= precioTp:
                    # Si se toca en apertura, usar ROI de apertura
                    if Open >= precioTp: return ('sltp', calcularRoi(Open, precioRef, sentido))
                    # Se toca durante la vela, usar TP teorico
                    else: return ('sltp', tp)
            # short           
            else:  
                
                precioTp = precioRef * (1 - tp/100)
                
                # Verificar si se toca TP
                if Open <= precioTp or Low <= precioTp:
                    # Si se toca en apertura, usar ROI de apertura
                    if Open <= precioTp: return ('sltp', calcularRoi(Open, precioRef, sentido))
                    # Se toca durante la vela, usar TP teórico
                    else: return ('sltp', tp)
    
        # SALIDA NORMAL - calcular ROI al cierre
        return ('normal', calcularRoi(Close, precioRef, sentido))

    
    
    
    df['ROID'] = 0
    df['ROIACUM'] = 0  
    
    df['precioRef']  = 0
    df['tipoSalida'] = ''
    
    df = df.reset_index(drop=False)
    
    if tp or sl:
    
        for i in df.index:
            
            estado = df.loc[i, 'TRADE']
    
            O = df.loc[i, 'Open']
            H = df.loc[i, 'High']
            L = df.loc[i, 'Low']
            C = df.loc[i, 'Close']
           
            if i > 0:
                # sentido long
                if   estado == 'In'   : 
    
                    df.loc[i,'tradePnl'] = 'In'
                    
                    precioRef = df.loc[i, 'Open']
                    salida, roi = pnlSalida(O,H,L,C, precioRef, sentido, tp, sl)

                    # solo lo reflejo en al apertura de la posicoin In
                    coste = (comision + slippage)
                    roi -= coste
    
                    # tanto salida normal comum sl y tp
                    df.loc[i, 'ROID']       = roi
                    df.loc[i, 'ROIACUM']    = roi
                    df.loc[i, 'precioRef']  = precioRef
                    df.loc[i, 'tipoSalida'] = salida
    
                elif estado in ['p','Out']:
    
                    # p y Out solo puede tener antes In o p, si no es huerfana y se elimina
                    if df.loc[i-1, 'tradePnl'] not in ['In','p']: 
                        
                        df.loc[i,'tradePnl'] = ''
                        
                        df.loc[i, 'ROID']       = 0
                        df.loc[i, 'ROIACUM']    = 0
                        df.loc[i, 'precioRef']  = 0
                        df.loc[i, 'tipoSalida'] = ''
    
                    # si antes es un In o p
                    else:
                        # una salida por sltp en la anterior
                        if df.loc[i-1,'tipoSalida'] == 'sltp':
    
                            df.loc[i,'tradePnl'] = 'Out'
                            
                            df.loc[i, 'ROID']    = 0
                            df.loc[i, 'ROIACUM'] = df.loc[i-1, 'ROIACUM']
    
                            df.loc[i, 'precioRef']  = 0
                            df.loc[i, 'tipoSalida'] = ''
                        
                        
                        # anterior es normal
                        else:
                            precioRef = df.loc[i-1, 'precioRef']
                            salida, roiRef = pnlSalida(O,H,L,C, precioRef, sentido, tp, sl)
    
                            # salida normal, no se tocan sl o tp
                            if salida == 'normal':
    
                                df.loc[i,'tradePnl'] = estado
                                
                                if estado == 'p': df.loc[i, 'ROID']       = (df.loc[i, 'Close'] / df.loc[i-1, 'Close'] - 1) * 100
                                if estado == 'Out': df.loc[i, 'ROID']     = (df.loc[i, 'Open'] / df.loc[i-1, 'Close'] - 1) * 100
                                    
                                df.loc[i, 'ROIACUM']    = roiRef
                                df.loc[i, 'precioRef']  = precioRef
                                df.loc[i, 'tipoSalida'] = salida
    
                            # se tocan sl o tp
                            else:
    
                                df.loc[i,'tradePnl'] = 'Out'
    
                                df.loc[i, 'ROID'] = ((1+roiRef/100) / (1+ df.loc[i-1, 'ROIACUM']/100) - 1) * 100
                                df.loc[i, 'ROIACUM'] = roiRef
                                
                                df.loc[i, 'precioRef']  = 0
                                df.loc[i, 'tipoSalida'] = salida
    
                # Cuando esta vacia
                else:
                    
                    df.loc[i,'tradePnl'] = ''
                    
                    df.loc[i, 'ROID']       = 0
                    df.loc[i, 'ROIACUM']    = 0
                    df.loc[i, 'precioRef']  = 0
                    df.loc[i, 'tipoSalida'] = ''
                
    
        df['TRADE'] = df.tradePnl
        
        

    # tanto el sl como el tp son 0, solo queda calcular el ROID y el ROIACUM
    # df.TRADE ya tiene los valores correctos de df.tradeVela
    else:
        # solo lo reflejo en al apertura de la posicoin In
        coste = (comision + slippage)
        for i in df.index:
            
            if i > 0:
                    # sentido long
                    estado = df.loc[i, 'TRADE']
                
                    if   estado == 'In'   :
                        df.loc[i, 'ROID']    = (df.loc[i, 'Close'] / df.loc[i, 'Open'] -1) * 100 - coste
                        df.loc[i, 'ROIACUM'] = df.loc[i, 'ROID']
                        
                    elif estado == 'p'    :
                        df.loc[i, 'ROID']    = (df.loc[i, 'Close'] / df.loc[i-1, 'Close'] - 1) * 100
                        df.loc[i, 'ROIACUM'] = ((1+df.loc[i, 'ROID']/100) * (1+df.loc[i-1, 'ROIACUM']/100) -1)*100
                        
                    elif estado == 'Out'  : 
                        df.loc[i, 'ROID']    = (df.loc[i, 'Open']  / df.loc[i-1, 'Close']  - 1) * 100
                        df.loc[i, 'ROIACUM'] = ((1+df.loc[i, 'ROID']/100) * (1+df.loc[i-1, 'ROIACUM']/100) -1)*100
                        
                    else                  : 
                        df.loc[i, 'ROID']    = 0
                        df.loc[i, 'ROIACUM'] = 0

        if sentido != 'long':
            df[ 'ROID']    = -df[ 'ROID']
            df['ROIACUM']  = -df['ROIACUM']
            

    df.set_index('Date', inplace=True)

    ''' '''

    # PARA COMROBAR QUE EL ROID CALCULADO POR SL O TP TIENE SENTIDO
    # roiComprobar es el roiD calculado a partir de tradeVela

    df['roiComprobar'] = 0

    df = df.reset_index(drop=False)
    for i in df.index:
        
        estado = df.loc[i, 'tradeVela']
      
        if i > 0:
    
            if estado   == 'In'   : df.loc[i, 'roiComprobar'] = (df.loc[i, 'Close'] / df.loc[i, 'Open'] -1) * 100
            elif estado == 'p'    : df.loc[i, 'roiComprobar'] = (df.loc[i, 'Close'] / df.loc[i-1, 'Close'] - 1) * 100
            elif estado == 'Out'  : df.loc[i, 'roiComprobar'] = (df.loc[i, 'Open']  / df.loc[i-1, 'Close']  - 1) * 100
            else                  : df.loc[i, 'roiComprobar'] = 0
    
    df.set_index('Date', inplace=True)
    
    return df

def dameGraficoSistema(df, numVelas, perSma, perRsi, rsiIn, rsiOut):

    dg = df.tail(numVelas).copy()

    fig, axes = plt.subplots(2, 1, sharex=True, figsize=(10, 8), gridspec_kw={'height_ratios': [2, 1]})
    
    # --- Primer subplot Precio ---
    ax0 = axes[0]
    
    ax0.plot(dg.index, dg['Close'], color='black', label='Close')
    ax0.plot(dg.index, dg[f's{perSma}'], color = 'blue', linestyle='--', label = 'Sma200', alpha=0.5)
    
    # Agregar puntos verdes y rojos
    dateInTrade  = dg.index[dg['TRADE'] == 'In']
    dateOutTrade = dg.index[dg['TRADE'] == 'Out']
    ax0.scatter(dateInTrade , dg.loc[dateInTrade, 'Close'], color='green', label='In', marker='o', alpha=0.5)
    ax0.scatter(dateOutTrade, dg.loc[dateOutTrade,'Close'], color='red', label='Out', marker='o', alpha=0.5)
    
    ax0.set_title('GRÁFICO DE PRECIO') 
    ax0.set_ylabel('Close')
    ax0.legend(loc='upper left')
    ax0.grid(True)
    
    # --- Segundo subplot Indicador ---
    ax1 = axes[1]
    
    ax1.plot(dg.index, dg[f'rsi{perRsi}'], color='gray', label='rsi2')
    
    ax1.axhline(90, color='red', linestyle='--', linewidth=0.8)
    ax1.axhline(rsiOut, color='black', linestyle='--', linewidth=0.8, alpha=0.5)
    ax1.axhline(rsiIn , color='green', linestyle='--', linewidth=0.8, alpha=0.5)
    
    ax1.set_title('INDICADORES') 
    ax1.set_ylabel('rsi2')
    ax1.set_xlabel('Date') 
    ax1.legend(loc='upper left')
    #ax1.grid(True) 
    
    # Ajustar el layout para evitar solapamientos entre titulos y ejes
    plt.tight_layout()

    plt.show()

    return

In [ ]:
data = pd.read_excel('dfIS.xlsx', index_col=0)

data = ocpSma(data, perSma)
data = ocpRsi(data,perRsi, True)

data = dameSistema(data, perSma, perRsi, rsiIn, rsiOut)
data = damePosition(data)
data = dameSalidaVelas(data, salidaVelas)
data = dameSalidaPnl(data, 'long', tp, sl, comision, slippage)

dameGraficoSistema(data, 1000, perSma, perRsi, rsiIn, rsiOut)

data.round(2).tail(300)

## BACTESTING RESULTADO

In [ ]:
def calculaCurvas(df, size):

    df['roiActivo'] = df.Close.pct_change()
    df['roiActivo'].iloc[0] = 0
    df['cvAct']  = 100 * (1+df.roiActivo).cumprod()

    df['roiActivo'].iloc[0] = 0
    # Multiplicado por el tamaño de posicion, aqui es 1
    df['cvSis']   =  100 * (1+df.ROID*size/100).cumprod()
    
    df['ddAct'] = ((df.cvAct/df.cvAct.cummax()-1)*100).round(2) 
    df['ddSis'] = ((df.cvSis/df.cvSis.cummax()-1)*100).round(2)

    return df

def crearDfBacktesting():
    
    dictBactesting = {'SYS': [], 'Y': [], 
                      'op': [],'pos': [], 'neg':[], 
                      'op/Y': [],'mDIT': [], 'tInv%':[],
                      'pa%':[], 
                      'capIn':[], 'capFn':[],
                      'roi%':[], 'cagr%': [], 
                      'mPos%': [],'mNeg%': [],
                      'em%':[], 'exca%':[],
                      'PF':[],'Payf':[],'shs':[],
                      'maxDD%': [], 'medDD%': [], 'OCP': []}
    
    
    dfBacktesting = pd.DataFrame(dictBactesting, index=[])
    
    dfBacktesting.set_index('SYS', drop = True, inplace = True)
    
    return dfBacktesting


def backActivoList(data, nombre = 'Activo'):
    
    df = data.copy()
    
    years = ((df.index.to_list()[-1] - df.index.to_list()[0]).days)/365
    
    op =  '-'
    pos = '-'
    neg = '-'
    
    opY = '-'
    
    DIT = len(df.index)
    
    tInv  = 100
    
    pa = '-'
    
    capIn = int(df.cvAct[0])
    capFn = int(round(df.cvAct[-1]))
    
    roi = int((df.cvAct[-1]/df.cvAct[0] -1)*100)
    cagr = round(((df.cvAct[-1]/df.cvAct[0]) ** (1 / years)  - 1)*100,2)
    
    mPos = '-'
    mNeg = '-'
    
    em  = '-'
    exca = '-'
    PF = '-'
    Payf = '-'
    shs = '-'
    
    maxDD = df.ddAct.min()
    medDD = df.ddAct.mean()
    
    OCP = abs(round(cagr/medDD,2))

    listaBacktesting = [nombre, round(years,2), 
                        op, pos, neg, 
                        opY, str(int(DIT)),str(int(tInv)),
                        pa, 
                        str(capIn), str(capFn),
                        str(roi), str(cagr),
                        mPos, mNeg, 
                        em, exca,
                        PF,Payf,shs,
                        str(int(maxDD)), str(int(medDD)), str(OCP)]
    
    return listaBacktesting


def backSistemaListOLD(data, nombre = 'Sistema'):
    
    df = data.copy()
     
    years = ((df.index[-1] - df.index[0]).days) / 365

    
    op =  len(df.loc[ df['TRADE'] == 'Out'])
    pos = len(df.loc[(df['TRADE'] == 'Out') & (df['ROIACUM'] >= 0)])
    neg = len(df.loc[(df['TRADE'] == 'Out') & (df['ROIACUM'] < 0)])

    opY = op / years

   
    #mDIT = len(df.loc[(df['TRADE'] == 'In') | (df['TRADE'] == 'p')]) / op
    if op > 0: mDIT = len(df.loc[(df['TRADE'] == 'In') | (df['TRADE'] == 'p')]) / op
    else:      mDIT = 0
        
    tInv = len(df.loc[(df['TRADE'] == 'In') | (df['TRADE'] == 'p')])/ len(df.index) * 100
   

    try:
        pa = round(pos / op * 100, 2)
    except ZeroDivisionError:
        pa = 0

    capIn = int(round(df['cvSis'].iloc[0]))
    capFn = int(round(df['cvSis'].iloc[-1]))

    roi = int(round((capFn / capIn - 1) * 100))
    cagr = round(((capFn / capIn) ** (1 / years) - 1) * 100,2)

    if pos: mPos = round(df.loc[(df['TRADE'] == 'Out') & (df['ROIACUM'] >= 0), 'ROIACUM'].mean(),2)
    else  : mPos = 0
        
    if neg : mNeg = round(df.loc[(df['TRADE'] == 'Out') & (df['ROIACUM'] <  0), 'ROIACUM'].mean(),2)
    else   : mNeg = 0
    
    em   = round(df.loc[(df['TRADE'] == 'Out'), 'ROIACUM'].mean(),2)

    sumPos = round(df.loc[(df['TRADE'] == 'Out') & (df['ROIACUM'] >= 0), 'ROIACUM'].sum(),2)
    sumNeg = round(df.loc[(df['TRADE'] == 'Out') & (df['ROIACUM'] <  0), 'ROIACUM'].sum(),2)
    

    # exca
    # Crear identificador de operacion: cada vez que aparece 'In' se incrementa el contador
    df['opId'] = (df['TRADE'] == 'In').cumsum()
    # Calcular la exca para cada operacion: minimo ROI acumulado diario o 0 si todos son positivos
    exca = df.groupby('opId')['ROIACUM'].min().apply(lambda x: x if x < 0 else 0).mean()

    PF    = -sumPos / sumNeg if sumNeg != 0 else 10
    Payf  = -mPos / mNeg if mNeg != 0 else 0
    shs   =  em / df.loc[(df['TRADE'] == 'Out'), 'ROIACUM'].std()

    maxDD = df['ddSis'].min()
    medDD = df['ddSis'].mean()
    
   
    if medDD: OCP = abs(cagr/medDD)
    else    : OCP = 10

    # Verificar y manejar NaN antes de convertir a int
    mDIT  = 0 if np.isnan(mDIT) else round(mDIT)
    tInv  = 0 if np.isnan(tInv) else round(tInv)
    capIn = 0 if np.isnan(capIn) else round(capIn)
    capFn = 0 if np.isnan(capFn) else round(capFn)

    em = 0 if pd.isna(em) else em
    
    PF = 0 if np.isnan(PF) else round(PF, 2)
    Payf = 0 if np.isnan(Payf) else round(Payf,2)
    shs = 0 if np.isnan(shs) else round(shs,2)
    
    maxDD = 0 if np.isnan(maxDD) else round(maxDD)
    medDD = 0 if np.isnan(medDD) else round(medDD)
    OCP   = 10 if np.isnan(OCP) else round(OCP, 2)
  
    listaBacktesting = [nombre, round(years,2), 
                       op, pos, neg, 
                       round(opY), mDIT,tInv,
                       round(pa), 
                       capIn, capFn,
                       roi, cagr,
                       mPos, mNeg, 
                       em, round(exca,2),
                       PF,Payf,shs,
                       maxDD, medDD, OCP]
        
        
    
    return listaBacktesting

def backSistemaList(data, nombre = 'Sistema'):
    
    df = data.copy()
     
    years = ((df.index[-1] - df.index[0]).days) / 365

    
    op =  len(df.loc[ df['TRADE'] == 'Out'])
    pos = len(df.loc[(df['TRADE'] == 'Out') & (df['ROIACUM'] >= 0)])
    neg = len(df.loc[(df['TRADE'] == 'Out') & (df['ROIACUM'] < 0)])

    opY = op / years

    if op > 0: mDIT = len(df.loc[(df['TRADE'] == 'In') | (df['TRADE'] == 'p')]) / op
    else:      mDIT = 0
        
    tInv = len(df.loc[(df['TRADE'] == 'In') | (df['TRADE'] == 'p')])/ len(df.index) * 100
   

    try:
        pa = round(pos / op * 100, 2)
    except ZeroDivisionError:
        pa = 0

    capIn = int(round(df['cvSis'].iloc[0]))
    capFn = int(round(df['cvSis'].iloc[-1]))

    roi = int(round((capFn / capIn - 1) * 100))
    cagr = round(((capFn / capIn) ** (1 / years) - 1) * 100,2)

    if pos: mPos = round(df.loc[(df['TRADE'] == 'Out') & (df['ROIACUM'] >= 0), 'ROIACUM'].mean(),2)
    else  : mPos = 0
        
    if neg : mNeg = round(df.loc[(df['TRADE'] == 'Out') & (df['ROIACUM'] <  0), 'ROIACUM'].mean(),2)
    else   : mNeg = 0
    
    em   = round(df.loc[(df['TRADE'] == 'Out'), 'ROIACUM'].mean(),2)

    sumPos = round(df.loc[(df['TRADE'] == 'Out') & (df['ROIACUM'] >= 0), 'ROIACUM'].sum(),2)
    sumNeg = round(df.loc[(df['TRADE'] == 'Out') & (df['ROIACUM'] <  0), 'ROIACUM'].sum(),2)
    

    # exca
    # Crear identificador de operacion: cada vez que aparece 'In' se incrementa el contador
    df['opId'] = (df['TRADE'] == 'In').cumsum()
    # Calcular la exca para cada operacion: minimo ROI acumulado diario o 0 si todos son positivos
    exca = df.groupby('opId')['ROIACUM'].min().apply(lambda x: x if x < 0 else 0).mean()

    # Profit Factor, si no hay perdidas, usar valor alto
    if sumNeg != 0:
        PF = -sumPos / sumNeg
    elif sumPos > 0:  # Hay ganancias pero no pérdidas
        PF = 99.99  # Valor alto que indica excelente performance
    else:
        PF = 0  # No hay operaciones o todas son neutras
    
    # Payoff -> si no hay perdidas promedio, usar valor alto
    if mNeg != 0:
        Payf = -mPos / mNeg
    elif mPos > 0:  # Hay ganancia promedio pero no perdida promedio
        Payf = 99.99  # Valor alto que indica excelente performance
    else:
        Payf = 0  # No hay operaciones o todas son neutras
    
    shs   =  em / df.loc[(df['TRADE'] == 'Out'), 'ROIACUM'].std()

    maxDD = df['ddSis'].min()
    medDD = df['ddSis'].mean()
    
    # OCP -> si no hay drawdown, usar valor alto
    if medDD != 0:
        OCP = abs(cagr/medDD)
    elif cagr > 0:  # Hay rendimiento pero no drawdown
        OCP = 99.99  # Valor alto que indica excelente performance
    else:
        OCP = 0  # No hay rendimiento

    # Verificar y manejar NaN antes de convertir a int
    mDIT  = 0 if np.isnan(mDIT) else round(mDIT)
    tInv  = 0 if np.isnan(tInv) else round(tInv)
    capIn = 0 if np.isnan(capIn) else round(capIn)
    capFn = 0 if np.isnan(capFn) else round(capFn)

    em = 0 if pd.isna(em) else em
    
    PF = 0 if np.isnan(PF) else round(PF, 2)
    Payf = 0 if np.isnan(Payf) else round(Payf,2)
    shs = 0 if np.isnan(shs) else round(shs,2)
    
    maxDD = 0 if np.isnan(maxDD) else round(maxDD)
    medDD = 0 if np.isnan(medDD) else round(medDD)
    OCP   = 0 if np.isnan(OCP) else round(OCP, 2)
  
    listaBacktesting = [nombre, round(years,2), 
                       op, pos, neg, 
                       round(opY), mDIT,tInv,
                       round(pa), 
                       capIn, capFn,
                       roi, cagr,
                       mPos, mNeg, 
                       em, round(exca,2),
                       PF,Payf,shs,
                       maxDD, medDD, OCP]    
    
    return listaBacktesting


def backAddList(dfBacktesting, listaBacktesting):
    
    dfBacktesting.reset_index(inplace=True, drop=False)
    
    dfBacktesting.loc[len(dfBacktesting)] = listaBacktesting
    
    dfBacktesting.set_index('SYS', drop = True, inplace = True)
    
    dfBacktesting = dfBacktesting.fillna('-')
    
    return dfBacktesting


def dameGraficoBacktest(df, velas, size):


    if velas == 0: dg = df.copy()
    else: dg = df.tail(velas).copy()

    fig, axes = plt.subplots(4, 1, sharex=True, figsize=(12, 12), gridspec_kw={'height_ratios': [2,1, 1, 2]})
    

    # --- PRIMER subplot Curvas Rendimiento---
    
    ax0 = axes[0]
    
    ax0.plot(dg.index, dg['cvAct'], color='gray', label='cvAct',alpha=0.1)
    ax0.plot(dg.index, dg['cvSis'], color='black', label='cvSis', alpha=0.5)

    ax0.axhline(100, color='black', linestyle='--', linewidth=0.8, alpha=0.5)
    
    ax0.set_title('CURVAS DE RENDIMIENTO') 
    ax0.set_ylabel('BASE 100')
    #ax0.set_xlabel('Date') 
    ax0.legend(loc='upper left')

    # --- SEGUNDO subplot DD---
    
    ax1 = axes[1]
    
    ax1.plot(dg.index, dg['ddAct'], color='gray', label='ddAct', alpha=0.1)
    ax1.plot(dg.index, dg['ddSis'], color='black', label='ddSis', alpha=0.5)

    ax1.axhline(0, color='black', linestyle='--', linewidth=0.8, alpha=0.5)
    
    ax1.set_title('DRAWDOWN') 
    ax1.set_ylabel('DRAWDOWN')
    ax1.set_xlabel('Date') 
    ax1.legend(loc='upper left')

    # --- TERCER subplot PnL---
    
    ax2 = axes[2]

    dateOpPos  = dg.index[(dg['TRADE'] == 'Out') & (dg['ROIACUM'] >= 0)]
    dateOpNeg  = dg.index[(dg['TRADE'] == 'Out') & (dg['ROIACUM'] <  0)]
    
    ax2.scatter(dateOpPos , dg.loc[dateOpPos, 'ROIACUM'] * size, color='green', label='Pos', marker='o', alpha=0.3)
    ax2.scatter(dateOpNeg , dg.loc[dateOpNeg, 'ROIACUM'] * size, color='red', label='Neg', marker='o', alpha=0.3)
    
    ax2.axhline(0, color='black', linestyle='--', linewidth=0.8, alpha=0.5)
    
    ax2.set_title('PnL OPERACIONES') 
    ax2.set_ylabel('PnL')
    #ax2.set_xlabel('Date') 
    ax2.legend(loc='upper left')

    # --- QUINTO subplot Señales ---
    
    ax3 = axes[3]
    
    ax3.plot(dg.index, dg['Close'], color='black', label='Close')
    #ax0.plot(dg.index, dg['s200'], color = 'blue', linestyle='--', label = 'Sma200', alpha=0.5)
    
    # Agregar puntos verdes y rojos
    dateInTrade  = dg.index[dg['TRADE'] == 'In']
    dateOutTrade = dg.index[dg['TRADE'] == 'Out']
    ax3.scatter(dateInTrade , dg.loc[dateInTrade, 'Close'], color='green', label='In', marker='o', alpha=0.5)
    ax3.scatter(dateOutTrade, dg.loc[dateOutTrade,'Close'], color='red', label='Out', marker='o', alpha=0.5)
    
    ax3.set_title('SEÑALES') 
    ax3.set_ylabel('Close')
    ax3.legend(loc='upper left')
    ax3.grid(True)
    
    
    # Ajustar el layout para evitar solapamientos entre titulos y ejes
    plt.tight_layout()

    plt.show()

    return
    

In [ ]:
perSma = 200
perRsi = 2

rsiIn  = 10
rsiOut = 50

salidaVelas = 0

sentido = 'long'

pSize = 1

tp = 0
sl = 0

# se aplica en la entrada para la entrada y la salida
# 0.01 es 0.01% se aplica a la entrada; $10000 -> $1
# 0.02 es 0.02% se aplica a la entrada; $10000 -> $2
comision = 0 
slippage = 0 

In [ ]:
print('Sma', perSma)
print('Rsi', perRsi, rsiIn, rsiOut)
print('Salida velas', salidaVelas)
print('Sentido', sentido)
print('pSize', pSize)
print('pnl', tp, sl)
print('coste', comision, slippage)

display(data.tail())

data = calculaCurvas(data, pSize)

dameGraficoBacktest(data, 0, pSize)


dfBacktesting = crearDfBacktesting()
listActivo = backActivoList(data)
listSistema = backSistemaList(data)

dfBacktesting = backAddList(dfBacktesting, listActivo)
dfBacktesting = backAddList(dfBacktesting, listSistema)

dfBacktesting

## BACKTESTING OPTIMIZACION

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def dameGraficoOpti(df, x, y, celda):
   
    pivotTabla = df.pivot(index = x, columns = y , values= celda)

    plt.figure(figsize=(14, 10))

    ax = sns.heatmap(pivotTabla,
                     annot=True,          # Mostrar valores en celdas
                     fmt='.2f',           # Formato de numeros
                     cmap='RdYlGn',       # Colormap (rojo-amarillo-verde)
                     center=0,            # Centrar en 0
                     cbar_kws={'label': celda},
                     linewidths=0.5)

    ax.tick_params(top=True, bottom=True, left=True, right=True,
                   labeltop=True, labelbottom=True, labelleft=True, labelright=True)

    # Etiquetas en todos los lados
    ax.set_xlabel(y, fontsize=12)
    ax.set_ylabel(x, fontsize=12)
    ax.xaxis.set_label_position('top')   
    ax.yaxis.set_label_position('right')  
    
    plt.title(f'Heatmap {celda}', fontsize=16, fontweight='bold')
    plt.xlabel(y, fontsize=12)
    plt.ylabel(x, fontsize=12)
    plt.tight_layout()
    plt.show()

    # Mejores combinaciones
    print(f"\nTop 5 mejores combinaciones por {celda}:")
    top5 = df.nlargest(5, celda)[[x, y, celda]]
    print(top5.to_string(index=False))

In [ ]:
print('Sma', perSma)
print('Rsi', perRsi, rsiIn, rsiOut)
print('Salida velas', salidaVelas)
print('Sentido', sentido)
print('pSize', pSize)
print('pnl', tp, sl)

datos = pd.read_excel('dfIS.xlsx', index_col=0)

data = datos.copy()

data = ocpSma(data, perSma)
data = ocpRsi(data,perRsi, True)
data = dameSistema(data, perSma, perRsi, rsiIn, rsiOut)
data = damePosition(data)
data = dameSalidaVelas(data, salidaVelas)
data = dameSalidaPnl(data, 'long', tp, sl, comision, slippage)

#dameGraficoSistema(data, 1000)

data = calculaCurvas(data, pSize)

#dameGraficoBacktest(data, 0, pSize)


dfBacktesting = crearDfBacktesting()
listActivo = backActivoList(data)
listSistema = backSistemaList(data)
dfBacktesting = backAddList(dfBacktesting, listActivo)
dfBacktesting = backAddList(dfBacktesting, listSistema)

display(dfBacktesting)

display(data.round(2).tail())

### Optimizando por Sma y Rsi

In [ ]:
perSma = 200
perRsi = 2

rsiIn  = 10
rsiOut = 50

salidaVelas = 0

sentido = 'long'

pSize = 1

tp = 0
sl = 0

# se aplica en la entrada para la entrada y la salida
# 0.01 es 0.01% se aplica a la entrada; $10000 -> $1
# 0.02 es 0.02% se aplica a la entrada; $10000 -> $2
comision = 0 
slippage = 0 

In [ ]:
print('Sma', perSma)
print('Rsi', perRsi, rsiIn, rsiOut)
print('Salida velas', salidaVelas)
print('Sentido', sentido)
print('pSize', pSize)
print('pnl', tp, sl)
print('coste', comision, slippage)

resultados = []

# Bucle de optimizacion
CONT = 0
for perSma in range(20, 410, 20):  # De 20 a 400 de 10 en 10
    for perRsi in range(2, 11):    # De 2 a 10 de 1 en 1

        data = datos.copy()

        data = ocpSma(data, perSma)
        data = ocpRsi(data, perRsi, True)
        data = data[data.index.year >= 2017]

        data = dameSistema(data, perSma, perRsi, rsiIn, rsiOut)
        data = damePosition(data)
        data = dameSalidaVelas(data, salidaVelas)
        data = dameSalidaPnl(data, sentido, tp, sl, comision, slippage)

        data = calculaCurvas(data, pSize)

        dfBacktesting = crearDfBacktesting()
        dfBacktesting['sma'] = perSma
        dfBacktesting['rsi'] = perRsi

        listSistema = backSistemaList(data)
        listSistema += [perSma, perRsi]

        dfBacktesting = backAddList(dfBacktesting, listSistema)

        resultados.append(dfBacktesting)
        
        CONT += 1
        if CONT % 10 ==0: print(f"Procesado: SMA={perSma}, RSI={perRsi}")

dfOpti = pd.concat(resultados, axis=0, ignore_index=True)

print(f"\nOptimizacion completada. Total combinaciones: {len(dfOpti)}")
display(dfOpti)

In [ ]:
dameGraficoOpti(dfOpti,'sma', 'rsi', 'op/Y')
dameGraficoOpti(dfOpti,'sma', 'rsi', 'pa%')
dameGraficoOpti(dfOpti,'sma', 'rsi', 'cagr%')
dameGraficoOpti(dfOpti,'sma', 'rsi', 'em%')
dameGraficoOpti(dfOpti,'sma', 'rsi', 'OCP')
dameGraficoOpti(dfOpti,'sma', 'rsi', 'medDD%')

In [ ]:
dfOpti.loc[(dfOpti['op/Y'] >= 7) & (dfOpti['pa%'] >= 70) & (dfOpti['cagr%'] >= 3) & (dfOpti['em%'] >= 0.20) & (dfOpti['maxDD%'] >= -15)]

### Optimizando Rsi In y Out

In [ ]:
perSma = 200
perRsi = 4

rsiIn  = 10
rsiOut = 50

salidaVelas = 0

sentido = 'long'

pSize = 1

tp = 0
sl = 0

# se aplica en la entrada para la entrada y la salida
# 0.01 es 0.01% se aplica a la entrada; $10000 -> $1
# 0.02 es 0.02% se aplica a la entrada; $10000 -> $2
comision = 0 
slippage = 0 

In [ ]:
from itertools import product

print('Sma', perSma)
print('Rsi', perRsi, rsiIn, rsiOut)
print('Salida velas', salidaVelas)
print('Sentido', sentido)
print('pSize', pSize)
print('pnl', tp, sl)
print('coste', comision, slippage)

resultados = []

# Bucle de optimizacion


rsiInPer  = range(5, 45, 5)
rsiOutPer = range(40, 75, 5)
CONT = 0
for rsiIn, rsiOut in product(rsiInPer, rsiOutPer): 
   
        data = datos.copy()
       
        data = ocpSma(data, perSma)
        data = ocpRsi(data, perRsi, True)
        data = data[data.index.year >= 2017]

        data = dameSistema(data, perSma, perRsi, rsiIn, rsiOut)
        data = damePosition(data)
        data = dameSalidaVelas(data, salidaVelas)
        data = dameSalidaPnl(data, sentido, tp, sl, comision, slippage)

        data = calculaCurvas(data, pSize)

        dfBacktesting = crearDfBacktesting()
        dfBacktesting['rsiIn']  = perSma
        dfBacktesting['rsiOut'] = perRsi

        listSistema = backSistemaList(data)
        listSistema += [rsiIn, rsiOut]

        dfBacktesting = backAddList(dfBacktesting, listSistema)

        resultados.append(dfBacktesting)
    
        CONT += 1
        if CONT % 10 ==0: print(f"Procesado: rsiIn {rsiIn}, rsiOut {rsiOut}")

dfOpti = pd.concat(resultados, axis=0, ignore_index=True)

print(f"\nOptimizacion completada. Total combinaciones: {len(dfOpti)}")
display(dfOpti)

In [ ]:
dameGraficoOpti(dfOpti,'rsiIn', 'rsiOut', 'op/Y')
dameGraficoOpti(dfOpti,'rsiIn', 'rsiOut', 'pa%')
dameGraficoOpti(dfOpti,'rsiIn', 'rsiOut', 'cagr%')
dameGraficoOpti(dfOpti,'rsiIn', 'rsiOut', 'em%')
dameGraficoOpti(dfOpti,'rsiIn', 'rsiOut', 'OCP')
dameGraficoOpti(dfOpti,'rsiIn', 'rsiOut', 'maxDD%')
dameGraficoOpti(dfOpti,'rsiIn', 'rsiOut', 'medDD%')

In [ ]:
dfSelec = dfOpti.loc[(dfOpti['op/Y'] >= 12) & (dfOpti['pa%'] >= 70) & (dfOpti['cagr%'] >= 3) & (dfOpti['em%'] >= 0.20) & (dfOpti['maxDD%'] >= -15)]
dfSelec

In [ ]:
dfSelec = top5 = dfSelec.nlargest(5, 'OCP')
dfSelec

### -> Eleccion de parametros

In [ ]:
perSma = 200 
perRsi = 4

rsiIn  = 40
rsiOut = 70

salidaVelas = 0

sentido = 'long'

pSize = 1

tp = 0
sl = 0

comision = 0
slippage = 0

# [perSma, perRsi, rsiIn, rsiOut, salidaVelas, sentido, pSize tp, sl, comision, slippage]
variacionesSistema = [[200, 4, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 4, 10, 50, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 10, 50, 0, 'long',1, 0, 0, 0, 0]
                     ]

In [ ]:
print('Sma', perSma)
print('Rsi', perRsi, rsiIn, rsiOut)
print('Salida velas', salidaVelas)
print('Sentido', sentido)
print('pSize', pSize)
print('pnl', tp, sl)
print('coste', comision, slippage)

datos = pd.read_excel('dfIS.xlsx', index_col=0)

data = datos.copy()

data = ocpSma(data, perSma)
data = ocpRsi(data,perRsi, True)
data = data[data.index.year >= 2017]

data = dameSistema(data, perSma, perRsi, rsiIn, rsiOut)
data = damePosition(data)
data = dameSalidaVelas(data, salidaVelas)
data = dameSalidaPnl(data, sentido, tp, sl, comision, slippage)

dameGraficoSistema(data, 1000, perSma, perRsi, rsiIn, rsiOut)

data = calculaCurvas(data, pSize)

dameGraficoBacktest(data, 0, pSize)

dfBacktesting = crearDfBacktesting()
listActivo    = backActivoList(data)
listSistema   = backSistemaList(data)
dfBacktesting = backAddList(dfBacktesting, listActivo)
dfBacktesting = backAddList(dfBacktesting, listSistema)

display(dfBacktesting)

display(data.round(2).tail(50))

## BACKTESTING VALIDACION

* La Validacion es un problema de datos:
  * Coges muchos datos -> Poco relevantes
  * Coges pocos datos -> Necesitas mas datos
  * Necesario inventar datos -> Datos sinteticos
* PRUEBAS DE VALIDACION:
  * VALIDACION IS
    * (1)  -> Portabilidad Activos
    * (2)  Portabilidad Timeframe
    * (3)  Montecarlo -> Admitir que no existe regimen de mercado -> Desnaturaliza los datos
    * (4)  Montecarlo Baraja Bloques
  * VALIDACION OS
    * (1)  -> OS
    * (2)  -> OS + Comisiones y Slippage (Importante si timeframe < D)
    * (3)  -> WALK FORWARD (ventana expansiva) ALL (IS + OS) para Sistemas dinamicos
    * (4)  -> CROSS VALIDATION (ventana rolling) ALL (IS + OS) para Sistemas dinamicos
  * VALICACION MKT
    * (1)  -> Prueba en paper (simulado) -> Software del Sistema TIEMPO REAL
    * (2) -> Prueba con poco capital -> Software del Sistema TIEMPO REAL
* ERRORES
  * Hacer Optimizacion con todos los datos (IS + OS) y pasar directamente a real

### VALIDACION IS

#### (1) -> Portabilidad Activos

In [ ]:
datos = pd.read_excel('dfIS.xlsx', index_col=0)

display(datos.head(1))
display(datos.tail(1))

In [ ]:
data = yf.download('AAPL', start="2015-05-18", end="2021-12-31", multi_level_index=False)
data.tail()

In [ ]:
perSma = 200 
perRsi = 4

rsiIn  = 40
rsiOut = 70

salidaVelas = 0

sentido = 'long'

pSize = 1

tp = 0
sl = 0

comision = 0
slippage = 0

# [perSma, perRsi, rsiIn, rsiOut, salidaVelas, sentido, pSize tp, sl, comision, slippage]
variacionesSistema = [[200, 4, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 4, 10, 50, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 10, 50, 0, 'long',1, 0, 0, 0, 0]
                     ]

In [ ]:
# SETUP ---------------------------------------------------
print('Sma', perSma)
print('Rsi', perRsi, rsiIn, rsiOut)
print('Salida velas', salidaVelas)
print('Sentido', sentido)
print('pSize', pSize)
print('pnl', tp, sl)
print('coste', comision, slippage)

# DATA ---------------------------------------------------

tickers = ['SPY', 'QQQ', 'TQQQ', 'IWM',
           'XLB', 'XLC', 'XLE','XLF', 'XLI', 'XLK','XLP','XLRE', 'XLU', 'XLV','XLY',
           'MSFT', 'AAPL', 'V', 'WMT', 'MMM',
           'GLD',
           'TLT',
           'EURUSD=X', 'GBPUSD=X',
           'BTC-USD', 'ETH-USD']

for t in tickers:

    print(f'\n\n********************** {t} **********************\n\n')

    data = yf.download(t, start="2015-05-18", end="2021-12-31", multi_level_index=False)

    # SISTEMA ---------------------------------------------------
    data = ocpSma(data, perSma)
    data = ocpRsi(data,perRsi, True)
    
    data = data[data.index.year >= 2017]
    
    data = dameSistema(data, perSma, perRsi, rsiIn, rsiOut)
    data = damePosition(data)
    data = dameSalidaVelas(data, salidaVelas)
    data = dameSalidaPnl(data, sentido, tp, sl, comision, slippage)
    
    #dameGraficoSistema(data, 1000, perSma, perRsi, rsiIn, rsiOut)
    
    #data.round(2).tail(50)
    
    
    display(data.tail())
    
    # BACKTESTING ---------------------------------------------------
    
    data = calculaCurvas(data, pSize)
    
    dameGraficoBacktest(data, 0, pSize)
    
    
    dfBacktesting = crearDfBacktesting()
    listActivo = backActivoList(data)
    listSistema = backSistemaList(data)
    
    dfBacktesting = backAddList(dfBacktesting, listActivo)
    dfBacktesting = backAddList(dfBacktesting, listSistema)
    
    display(dfBacktesting)



#### (2) Portabilidad Timeframe

In [ ]:
datos = pd.read_excel('dfIS.xlsx', index_col=0)
datos.tail(20)

In [ ]:
datos = datos.resample('W').last()
datos.tail(20)

In [ ]:
perSma = 200 
perRsi = 4

rsiIn  = 40
rsiOut = 70

salidaVelas = 0

sentido = 'long'

pSize = 1

tp = 0
sl = 0

comision = 0
slippage = 0

# [perSma, perRsi, rsiIn, rsiOut, salidaVelas, sentido, pSize tp, sl, comision, slippage]
variacionesSistema = [[200, 4, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 4, 10, 50, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 10, 50, 0, 'long',1, 0, 0, 0, 0]
                     ]

In [ ]:
print('Sma', perSma)
print('Rsi', perRsi, rsiIn, rsiOut)
print('Salida velas', salidaVelas)
print('Sentido', sentido)
print('pSize', pSize)
print('pnl', tp, sl)
print('coste', comision, slippage)

data = datos.copy()

# SISTEMA ---------------------------------------------------

data = ocpSma(data, perSma)
data = ocpRsi(data,perRsi, True)
data = data[data.index.year >= 2017]

data = dameSistema(data, perSma, perRsi, rsiIn, rsiOut)
data = damePosition(data)
data = dameSalidaVelas(data, salidaVelas)
data = dameSalidaPnl(data, sentido, tp, sl, comision, slippage)

dameGraficoSistema(data, 1000, perSma, perRsi, rsiIn, rsiOut)

# BACKTESTING ---------------------------------------------------

data = calculaCurvas(data, pSize)

dameGraficoBacktest(data, 0, pSize)

dfBacktesting = crearDfBacktesting()
listActivo    = backActivoList(data)
listSistema   = backSistemaList(data)
dfBacktesting = backAddList(dfBacktesting, listActivo)
dfBacktesting = backAddList(dfBacktesting, listSistema)

display(dfBacktesting)

#display(data.round(2).tail(200))
data

#### (3) Montecarlo

In [ ]:
datos = pd.read_excel('dfIS.xlsx', index_col=0)
display(datos.tail())


datos.rename(columns={'Close': 'CloseOriginal'}, inplace=True)

datos['roiD'] = datos.CloseOriginal.pct_change()

datos['Close'] = np.nan
precioInicial = datos['CloseOriginal'].iloc[0]
datos['Close'].iloc[0] = precioInicial

# frac 1 es el 100 de los datos | 35 es la semilla del generador aleatorio
#retornosBaraja = datos.roiD.dropna().sample(frac=1, random_state=35).reset_index(drop=True)
retornosMezcla = datos.roiD.dropna().sample(frac=1).reset_index(drop=True)
precioSintetico = precioInicial * (1 + retornosMezcla).cumprod()


datos['Close'].iloc[1:] = precioSintetico.values


display(datos.head())

datos.CloseOriginal.plot()
datos.Close.plot()

In [ ]:
def mezclaDataC(df, seed = None):

    df.rename(columns={'Close': 'CloseOriginal'}, inplace=True)

    df['roiC'] = df.CloseOriginal.pct_change()
    
    df['Close'] = np.nan
    precioInicial       = df['CloseOriginal'].iloc[0]
    df['Close'].iloc[0] = precioInicial
    
    # frac 1 es el 100 de los datos | 35 es la semilla del generador aleatorio
    retornosMezcla = df.roiC.dropna().sample(frac=1, random_state= seed).reset_index(drop=True)
    precioSintetico = precioInicial * (1 + retornosMezcla).cumprod()
    
    
    df['Close'].iloc[1:] = precioSintetico.values

    return df


def mezclaDataOHLC(df, seed=None):

    'Barajeamos dias completos, no O H L C cada uno por su lado'
    
    df.rename(columns={'Close': 'CloseOriginal',
                       'High': 'HighOriginal', 
                       'Low': 'LowOriginal',
                       'Open': 'OpenOriginal'}, 
              inplace=True)

    # Calcula SOLO retorno del Close
    df['roiC'] = df.CloseOriginal.pct_change()
    
    # Calcula ratios intradiarios (relaciones dentro del dia) Guardar coherencia de cada vela
    df['ratioHC'] = df.HighOriginal / df.CloseOriginal    # High vs Close del mismo dia
    df['ratioLC'] = df.LowOriginal / df.CloseOriginal     # Low vs Close del mismo dia  
    df['ratioOC'] = df.OpenOriginal / df.CloseOriginal    # Open vs Close del mismo dia

    # Crear DataFrame con datos validos
    datosCompletos = df[['roiC', 'ratioHC', 'ratioLC', 'ratioOC']].dropna()
    
    # Barajear filas completas (dias completos)
    datosMezclados = datosCompletos.sample(frac=1, random_state=seed).reset_index(drop=True)

    # Inicializa columnas sinteticas
    for col in ['Close', 'High', 'Low', 'Open']: 
        df[col] = np.nan

    # Precios iniciales
    df['Close'].iloc[0] = df['CloseOriginal'].iloc[0]
    df['High'].iloc[0]  = df['HighOriginal'].iloc[0]
    df['Low'].iloc[0]   = df['LowOriginal'].iloc[0]
    df['Open'].iloc[0]  = df['OpenOriginal'].iloc[0]

    # Generar precios sinteticos manteniendo coherencia OHLC
    for i in range(len(datosMezclados)):
        
        idx = i + 1
        
        # Evoluciona Close con retorno barajeado
        nuevoClose = df['Close'].iloc[idx-1] * (1 + datosMezclados['roiC'].iloc[i])
        df['Close'].iloc[idx] = nuevoClose
        
        # Calcula H, L, O basados en ratios del dia barajeado
        df['High'].iloc[idx] = nuevoClose * datosMezclados['ratioHC'].iloc[i]
        df['Low'].iloc[idx]  = nuevoClose * datosMezclados['ratioLC'].iloc[i]
        df['Open'].iloc[idx] = nuevoClose * datosMezclados['ratioOC'].iloc[i]

    return df

In [ ]:
datos = pd.read_excel('dfIS.xlsx', index_col=0)
display(datos.tail())

datos = mezclaDataOHLC(datos)

display(datos.head(5))
display(datos.tail(5))

datos.CloseOriginal.plot()
datos.Close.plot()

In [ ]:
perSma = 200 
perRsi = 4

rsiIn  = 40
rsiOut = 70

salidaVelas = 0

sentido = 'long'

pSize = 1

tp = 0
sl = 0

comision = 0
slippage = 0

# [perSma, perRsi, rsiIn, rsiOut, salidaVelas, sentido, pSize tp, sl, comision, slippage]
variacionesSistema = [[200, 4, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 4, 10, 50, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 10, 50, 0, 'long',1, 0, 0, 0, 0]
                     ]

In [ ]:
# SETUP ---------------------------------------------------
print('Sma', perSma)
print('Rsi', perRsi, rsiIn, rsiOut)
print('Salida velas', salidaVelas)
print('Sentido', sentido)
print('pSize', pSize)
print('pnl', tp, sl)
print('coste', comision, slippage)

# DATA ---------------------------------------------------
datos = pd.read_excel('dfIS.xlsx', index_col=0)

resultadosMontecarlo = []
dfCurvas = pd.DataFrame()  # DataFrame para todas las curvas

for i in range(10):
    print(f'********************** Montecarlo {i} **********************')
    
    data = datos.copy()
    data = mezclaDataOHLC(data, 42+i)
    
    # SISTEMA ---------------------------------------------------
    data = ocpSma(data, perSma)
    data = ocpRsi(data,perRsi, True)
    
    data = data[data.index.year >= 2017]
    
    data = dameSistema(data, perSma, perRsi, rsiIn, rsiOut)
    data = damePosition(data)
    data = dameSalidaVelas(data, salidaVelas)
    data = dameSalidaPnl(data, 'long', tp, sl, comision, slippage)
    
    dameGraficoSistema(data, 1000, perSma, perRsi, rsiIn, rsiOut)
    display(data.tail())
    
    # BACKTESTING ---------------------------------------------------
    data = calculaCurvas(data, pSize)
    dameGraficoBacktest(data, 0, pSize)
    
    # RECOGER CURVAS ---------------------------------------------------
    if i == 0:
        # En la primera iteracion, guardamos tambien la curva del activo
        dfCurvas['cvAct'] = data['cvAct']
    
    # Siempre guardamos la curva del sistema de cada iteracion
    dfCurvas[f'cvSisIter{i}'] = data['cvSis']
    
    dfBacktesting = crearDfBacktesting()
    listActivo    = backActivoList(data)
    listSistema   = backSistemaList(data)
    
    dfBacktesting = backAddList(dfBacktesting, listActivo)
    dfBacktesting = backAddList(dfBacktesting, listSistema)
    
    display(dfBacktesting)
    
    # Recoger TODA la fila Sistema
    filaSistema = dfBacktesting.loc['Sistema'].to_dict()
    filaSistema['iteracion'] = i  # Añadir numero de iteracion
    
    resultadosMontecarlo.append(filaSistema)

# Crear DataFrame final con TODAS las columnas del sistema
dfMontecarlo = pd.DataFrame(resultadosMontecarlo)

# Reordenar columnas para que la columna iteracion este primera
cols = ['iteracion'] + [col for col in dfMontecarlo.columns if col != 'iteracion']
dfMontecarlo = dfMontecarlo[cols]

print('\n' + '='*80)
print('RESULTADOS COMPLETOS MONTECARLO - TODAS LAS METRICAS')
print("="*80)
display(dfMontecarlo)

# Estadisticas resumen de las metricas mas importantes
metricasClave = ['pa%', 'cagr%', 'roi%', 'medDD%', 'maxDD%', 'OCP', 'PF', 'op']
if all(col in dfMontecarlo.columns for col in metricasClave):
    
    print("\n" + "="*50)
    print("ESTADISTICAS RESUMEN - METRICAS CLAVE")
    print("="*50)
    display(dfMontecarlo[metricasClave].describe())

# GRAFICO DE TODAS LAS CURVAS ---------------------------------------------------
print('\n' + '='*50)
print('GRAFICO DE CURVAS MONTECARLO')
print('='*50)


plt.figure(figsize=(15, 10))

# Dibuja curva del activo (solo una vez)
plt.plot(dfCurvas.index, dfCurvas['cvAct'], 
         label='Activo', linewidth=3, color='black', alpha=0.9)

# Dibuja todas las curvas del sistema
colores = plt.cm.tab10(np.linspace(0, 1, 10))
for i in range(10):
    colName = f'cvSisIter{i}'
    if colName in dfCurvas.columns:
        plt.plot(dfCurvas.index, dfCurvas[colName], 
                label=f'Sistema Iter {i}', alpha=0.7, color=colores[i])

plt.title('Curvas de Rendimiento - Montecarlo vs Activo', fontsize=16, fontweight='bold')
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Valor de la Curva', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()


plt.show()


#### (4) Montecarlo Baraja Bloques

In [ ]:
def mezclaDataBloques(df, numBloques=4, seed=None):
    
    dfOriginal = df.copy()
    
    # Renombra columnas originales
    df.rename(columns={'Close': 'CloseOriginal',
                       'High': 'HighOriginal', 
                       'Low': 'LowOriginal',
                       'Open': 'OpenOriginal'}, 
              inplace=True)

    # Calcula datos para barajeo
    df['roiC']    = df.CloseOriginal.pct_change()
    df['ratioHC'] = df.HighOriginal / df.CloseOriginal
    df['ratioLC'] = df.LowOriginal / df.CloseOriginal  
    df['ratioOC'] = df.OpenOriginal / df.CloseOriginal

    datosCompletos = df[['roiC', 'ratioHC', 'ratioLC', 'ratioOC']].dropna()
    
    # Dividir en bloques
    tamBloque = len(datosCompletos) // numBloques
    bloquesBarajeados = []
    
    np.random.seed(seed)
    
    for i in range(numBloques):
        inicio = i * tamBloque
        if i == numBloques - 1:  # Ultimo bloque incluye el resto
            fin = len(datosCompletos)
        else:
            fin = (i + 1) * tamBloque
            
        bloque = datosCompletos.iloc[inicio:fin].copy()
        
        # Barajea cada bloque con semilla diferente
        bloqueBarajeado = bloque.sample(frac=1, random_state=seed+i).reset_index(drop=True)
        bloquesBarajeados.append(bloqueBarajeado)
    
    # Unir todos los bloques barajeados
    datosMezclados = pd.concat(bloquesBarajeados, ignore_index=True)

    # Generar precios sinteticos
    for col in ['Close', 'High', 'Low', 'Open']: 
        df[col] = np.nan

    # Precios iniciales
    df['Close'].iloc[0] = df['CloseOriginal'].iloc[0]
    df['High'].iloc[0]  = df['HighOriginal'].iloc[0]
    df['Low'].iloc[0]   = df['LowOriginal'].iloc[0]
    df['Open'].iloc[0]  = df['OpenOriginal'].iloc[0]

    # Aplica datos mezclados
    for i in range(len(datosMezclados)):
        idx = i + 1
        nuevoClose = df['Close'].iloc[idx-1] * (1 + datosMezclados['roiC'].iloc[i])
        df['Close'].iloc[idx] = nuevoClose
        df['High'].iloc[idx]  = nuevoClose * datosMezclados['ratioHC'].iloc[i]
        df['Low'].iloc[idx]   = nuevoClose * datosMezclados['ratioLC'].iloc[i]
        df['Open'].iloc[idx]  = nuevoClose * datosMezclados['ratioOC'].iloc[i]

    return df

In [ ]:
perSma = 200 
perRsi = 4

rsiIn  = 40
rsiOut = 70

salidaVelas = 0

sentido = 'long'

pSize = 1

tp = 0
sl = 0

comision = 0
slippage = 0

# [perSma, perRsi, rsiIn, rsiOut, salidaVelas, sentido, pSize tp, sl, comision, slippage]
variacionesSistema = [[200, 4, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 4, 10, 50, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 10, 50, 0, 'long',1, 0, 0, 0, 0]
                     ]

In [ ]:
# SETUP ---------------------------------------------------
print('Sma', perSma)
print('Rsi', perRsi, rsiIn, rsiOut)
print('Salida velas', salidaVelas)
print('Sentido', sentido)
print('pSize', pSize)
print('pnl', tp, sl)
print('coste', comision, slippage)

# DATA ---------------------------------------------------
datos = pd.read_excel('dfIS.xlsx', index_col=0)

resultadosMontecarlo = []
dfCurvas = pd.DataFrame()  # DataFrame para todas las curvas

for i in range(10):
    print(f'********************** Montecarlo Baraja Bloques {i} **********************')
    
    data = datos.copy()
    data = mezclaDataBloques(data, numBloques=5, seed=42+i)
    
    # SISTEMA ---------------------------------------------------
    data = ocpSma(data, perSma)
    data = ocpRsi(data,perRsi, True)
    
    data = data[data.index.year >= 2017]
    
    data = dameSistema(data, perSma, perRsi, rsiIn, rsiOut)
    data = damePosition(data)
    data = dameSalidaVelas(data, salidaVelas)
    data = dameSalidaPnl(data, 'long', tp, sl, comision, slippage)
    
    dameGraficoSistema(data, 1000, perSma, perRsi, rsiIn, rsiOut)
    display(data.tail())
    
    # BACKTESTING ---------------------------------------------------
    data = calculaCurvas(data, pSize)
    dameGraficoBacktest(data, 0, pSize)
    
    # RECOGE CURVAS ---------------------------------------------------
    if i == 0:
        # En la primera iteracion, guardamos tambien la curva del activo
        dfCurvas['cvAct'] = data['cvAct']
    
    # Siempre guardamos la curva del sistema de cada iteracion
    dfCurvas[f'cvSisIter{i}'] = data['cvSis']
    
    dfBacktesting = crearDfBacktesting()
    listActivo    = backActivoList(data)
    listSistema   = backSistemaList(data)
    
    dfBacktesting = backAddList(dfBacktesting, listActivo)
    dfBacktesting = backAddList(dfBacktesting, listSistema)
    
    display(dfBacktesting)
    
    # Recoge TODA la fila Sistema
    filaSistema = dfBacktesting.loc['Sistema'].to_dict()
    filaSistema['iteracion'] = i  # Añade numero de iteracion
    
    resultadosMontecarlo.append(filaSistema)

# Crea DataFrame final con TODAS las columnas del sistema
dfMontecarlo = pd.DataFrame(resultadosMontecarlo)

# Reordena columnas para que la columna iteracion este primera
cols = ['iteracion'] + [col for col in dfMontecarlo.columns if col != 'iteracion']
dfMontecarlo = dfMontecarlo[cols]

print('\n' + '='*80)
print('RESULTADOS COMPLETOS MONTECARLO BARAJA BLOQUES - TODAS LAS METRICAS')
print('='*80)
display(dfMontecarlo)

# Estadisticas resumen de las metricas mas importantes
metricasClave = ['pa%', 'cagr%', 'roi%', 'medDD%', 'maxDD%', 'OCP', 'PF', 'op']
if all(col in dfMontecarlo.columns for col in metricasClave):
    
    print('\n' + '='*50)
    print('ESTADISTICAS RESUMEN - METRICAS CLAVE')
    print("="*50)
    display(dfMontecarlo[metricasClave].describe())

# GRAFICO DE TODAS LAS CURVAS ---------------------------------------------------
print('\n' + '='*50)
print('GRAFICO DE CURVAS MONTECARLO BARAJA BLOQUES')
print('='*50)

#import matplotlib.pyplot as plt
#import numpy as np

plt.figure(figsize=(15, 10))

# Dibuja curva del activo (solo una vez)
plt.plot(dfCurvas.index, dfCurvas['cvAct'], 
         label='Activo', linewidth=3, color='black', alpha=0.9)

# Dibuja todas las curvas del sistema
colores = plt.cm.tab10(np.linspace(0, 1, 10))
for i in range(10):
    colName = f'cvSisIter{i}'
    if colName in dfCurvas.columns:
        plt.plot(dfCurvas.index, dfCurvas[colName], 
                label=f'Sistema Iter {i}', alpha=0.7, color=colores[i])

plt.title('Curvas de Rendimiento - Montecarlo Baraja Bloques vs Activo', fontsize=16, fontweight='bold')
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Valor de la Curva', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()


plt.show()


### VALIDACION OS

#### (1) -> OS

In [ ]:
datos = pd.read_excel('dfAll.xlsx', index_col=0)
display(datos.tail())

datos = pd.read_excel('dfOS.xlsx', index_col=0)
display(datos.head())

In [ ]:
perSma = 200 
perRsi = 4

rsiIn  = 40
rsiOut = 70

salidaVelas = 0

sentido = 'long'

pSize = 1

tp = 0
sl = 0

comision = 0
slippage = 0

# [perSma, perRsi, rsiIn, rsiOut, salidaVelas, sentido, pSize tp, sl, comision, slippage]
variacionesSistema = [[200, 4, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 4, 10, 50, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 10, 50, 0, 'long',1, 0, 0, 0, 0]
                     ]

In [ ]:
# SETUP ---------------------------------------------------
print('Sma', perSma)
print('Rsi', perRsi, rsiIn, rsiOut)
print('Salida velas', salidaVelas)
print('Sentido', sentido)
print('pSize', pSize)
print('pnl', tp, sl)
print('coste', comision, slippage)

# DATA ---------------------------------------------------

datos = pd.read_excel('dfAll.xlsx', index_col=0)

data = datos.copy()


    
# SISTEMA ---------------------------------------------------
data = ocpSma(data, perSma)
data = ocpRsi(data,perRsi, True)

data = data[data.index >= '2022-01-03']

data = dameSistema(data, perSma, perRsi, rsiIn, rsiOut)
data = damePosition(data)
data = dameSalidaVelas(data, salidaVelas)
data = dameSalidaPnl(data, sentido, tp, sl, comision, slippage)

#dameGraficoSistema(data, 1000, perSma, perRsi, rsiIn, rsiOut)

#data.round(2).tail(50)


display(data.tail(100))

# BACKTESTING ---------------------------------------------------

data = calculaCurvas(data, pSize)

dameGraficoBacktest(data, 0, pSize)


dfBacktesting = crearDfBacktesting()
listActivo = backActivoList(data)
listSistema = backSistemaList(data)

dfBacktesting = backAddList(dfBacktesting, listActivo)
dfBacktesting = backAddList(dfBacktesting, listSistema)

display(dfBacktesting)


#### (2) -> OS + Comisiones + Slippage

In [ ]:
perSma = 200 
perRsi = 4

rsiIn  = 40
rsiOut = 70

salidaVelas = 0

sentido = 'long'

pSize = 1

tp = 0
sl = 0

# 0.01 supone un $1 por cada $10k
# 0.03 como maximo, en diario es 0
comision = 0.01 
slippage = 0.03

# [perSma, perRsi, rsiIn, rsiOut, salidaVelas, sentido, pSize tp, sl, comision, slippage]
variacionesSistema = [[200, 4, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 4, 10, 50, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 10, 50, 0, 'long',1, 0, 0, 0, 0]
                     ]

In [ ]:
# SETUP ---------------------------------------------------
print('Sma', perSma)
print('Rsi', perRsi, rsiIn, rsiOut)
print('Salida velas', salidaVelas)
print('Sentido', sentido)
print('pSize', pSize)
print('pnl', tp, sl)
print('coste', comision, slippage)

# DATA ---------------------------------------------------

datos = pd.read_excel('dfAll.xlsx', index_col=0)

data = datos.copy()


    
# SISTEMA ---------------------------------------------------
data = ocpSma(data, perSma)
data = ocpRsi(data,perRsi, True)

data = data[data.index >= '2022-01-03']

data = dameSistema(data, perSma, perRsi, rsiIn, rsiOut)
data = damePosition(data)
data = dameSalidaVelas(data, salidaVelas)
data = dameSalidaPnl(data, 'long', tp, sl, comision, slippage)

#dameGraficoSistema(data, 1000, perSma, perRsi, rsiIn, rsiOut)

#data.round(2).tail(50)


display(data.tail(100))

# BACKTESTING ---------------------------------------------------

data = calculaCurvas(data, pSize)

dameGraficoBacktest(data, 0, pSize)


dfBacktesting = crearDfBacktesting()
listActivo = backActivoList(data)
listSistema = backSistemaList(data)

dfBacktesting = backAddList(dfBacktesting, listActivo)
dfBacktesting = backAddList(dfBacktesting, listSistema)

display(dfBacktesting)

#### (3) -> Walk Forward (ventana extensiva)

In [ ]:
perSma = 200 
perRsi = 4

rsiIn  = 40
rsiOut = 70

salidaVelas = 0

sentido = 'long'

pSize = 1

tp = 0
sl = 0

# 0.01 supone un $1 por cada $10k
# 0.03 como maximo, en diario es 0
comision = 0
slippage = 0

# [perSma, perRsi, rsiIn, rsiOut, salidaVelas, sentido, pSize tp, sl, comision, slippage]
variacionesSistema = [[200, 4, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 4, 10, 50, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 10, 50, 0, 'long',1, 0, 0, 0, 0]
                     ]

In [ ]:
def evaluarSistema(data, variacion, inicioEval=None, finEval=None):
    
    perSma, perRsi, rsiIn, rsiOut, salidaVelas, sentido, pSize, tp, sl, comision, slippage = variacion
    
    dataTest = data.copy()
    
    # Calcula indicadores con TODOS los datos
    dataTest = ocpSma(dataTest, perSma)
    dataTest = ocpRsi(dataTest, perRsi, True)
    dataTest = dameSistema(dataTest, perSma, perRsi, rsiIn, rsiOut)
    dataTest = damePosition(dataTest)
    dataTest = dameSalidaVelas(dataTest, salidaVelas)
    dataTest = dameSalidaPnl(dataTest, sentido, tp, sl, comision, slippage)
    
    # Si se especifica ventana, tomar solo esa parte para evaluacion
    if inicioEval is not None and finEval is not None:
        dataTest = dataTest.iloc[inicioEval:finEval].copy()
    
    # Calcula metricas solo en la ventana especificada
    dataTest = calculaCurvas(dataTest, pSize)
    
    dfBacktesting = crearDfBacktesting()
    listSistema = backSistemaList(dataTest)
    dfBacktesting = backAddList(dfBacktesting, listSistema)
    
    return dfBacktesting.loc['Sistema', 'OCP'], dfBacktesting

In [ ]:
# Cambiar por: 'OCP', 'PF', 'cagr%', 'pa%', 'roi%', etc.
metricaSeleccion = 'PF'  

numBloques = 5

In [ ]:
datos = pd.read_excel('dfAll.xlsx', index_col=0)
print(f'Datos originales: {datos.index[0]} a {datos.index[-1]} ({len(datos)} días)')

datos = datos[datos.index.year >= 2017].copy()
print(f'Datos desde 2017: {datos.index[0]} a {datos.index[-1]} ({len(datos)} días)')

print(f'METRICA DE SELECCION: {metricaSeleccion}')
print('='*50)

# WALK FORWARD VENTANA EXTENSIVA
resultadosWF = []
curvasValidacion = []  # Para recoger las curvas de cada validacion
tamBloque = len(datos) // numBloques

for bloque in range(numBloques):
    
    print(f'\n=============================================')
    print(f'BLOQUE {bloque+1}/{numBloques}')
    print(f'=============================================')
    
    # Definir INDICES de ventanas
    inicioTrain = 0 # por ser ventana expansiva
    finTrain = (bloque + 1) * tamBloque
    inicioVal = finTrain
    
    if bloque == numBloques - 1:
        finVal = len(datos)
    else:
        finVal = min(finTrain + tamBloque, len(datos))
    
    # Verificar que hay datos suficientes
    if inicioVal >= len(datos) or finVal - inicioVal < 50:
        print('No hay datos suficientes para validacion')
        continue
    
    print(f'TRAINING: {datos.index[inicioTrain]} a {datos.index[finTrain-1]} ({finTrain-inicioTrain} dias)')
    print(f'VALIDACION: {datos.index[inicioVal]} a {datos.index[finVal-1]} ({finVal-inicioVal} dias)')
    
    # OPTIMIZACION EN TRAINING
    print(f'\nEVALUANDO VARIACIONES EN TRAINING:')
    print('----------------------------------------')
    
    mejorVariacion = None
    mejorScore = -999
    mejorIndice = -1
    
    for i, variacion in enumerate(variacionesSistema):
        try:
            _, dfBacktestingTrain = evaluarSistema(datos, variacion, inicioTrain, finTrain)
            score = dfBacktestingTrain.loc['Sistema', metricaSeleccion]
            
            print(f'Variacion {i}: {variacion} → {metricaSeleccion}: {round(score,2)}')
            
            if score > mejorScore:
                mejorScore = score
                mejorVariacion = variacion
                mejorIndice = i
                
        except Exception as e:
            print(f'Variacion {i}: ERROR - {e}')
            continue
    
    print('-----------------------------------------------')
    print(f'MEJOR EN TRAINING: Variacion {mejorIndice} → {metricaSeleccion}: {mejorScore:.2f}')
    print(f'   Parametros: {mejorVariacion}')
    
    # APLICA MEJOR SISTEMA A VALIDACION
    if mejorVariacion is not None:
        try:
            print(f'\nAPLICANDO VARIACION {mejorIndice} EN VALIDACION...')
            scoreVal, dfBacktesting = evaluarSistema(datos, mejorVariacion, inicioVal, finVal)
            
            # Mostrar la metrica de validacion correspondiente
            metricaVal = dfBacktesting.loc['Sistema', metricaSeleccion]
            print(f'RESULTADO VALIDACION: {metricaSeleccion} {metricaVal:.2f} | OCP {scoreVal:.2f}')
            
            # OBTENER CURVAS APLICANDO EL SISTEMA COMPLETO A LA VALIDACION
            perSma, perRsi, rsiIn, rsiOut, salidaVelas, sentido, pSize, tp, sl, comision, slippage = mejorVariacion
            
            dataValidacion = datos.copy()
            dataValidacion = ocpSma(dataValidacion, perSma)
            dataValidacion = ocpRsi(dataValidacion, perRsi, True)
            dataValidacion = dameSistema(dataValidacion, perSma, perRsi, rsiIn, rsiOut)
            dataValidacion = damePosition(dataValidacion)
            dataValidacion = dameSalidaVelas(dataValidacion, salidaVelas)
            dataValidacion = dameSalidaPnl(dataValidacion, sentido, tp, sl, comision, slippage)
            dataValidacion = dataValidacion.iloc[inicioVal:finVal].copy()
            dataValidacion = calculaCurvas(dataValidacion, pSize)
            
            # RECOGER CURVAS DE VALIDACION
            curvasValidacion.append({
                'bloque': bloque + 1,
                'fechas': dataValidacion.index,
                'cvAct': dataValidacion['cvAct'],
                'cvSis': dataValidacion['cvSis'],
                'inicioVal': inicioVal,
                'finVal': finVal
            })
            
            resultadosWF.append({
                'bloque': bloque + 1,
                'training': f'{datos.index[inicioTrain].strftime("%Y-%m")} a {datos.index[finTrain-1].strftime("%Y-%m")}',
                'metricaTraining': mejorScore,
                'variacionElegida': mejorIndice,
                'parametros': mejorVariacion,
                'validation': f'{datos.index[inicioVal].strftime("%Y-%m")} a {datos.index[finVal-1].strftime("%Y-%m")}',
                'metricaVal': metricaVal, 
                'OCP': scoreVal,
                'op': dfBacktesting.loc['Sistema', 'op'],
                'Pa': dfBacktesting.loc['Sistema', 'pa%'],
                'CAGR': dfBacktesting.loc['Sistema', 'cagr%'],
                'medDD': dfBacktesting.loc['Sistema', 'medDD%'],
                'PF': dfBacktesting.loc['Sistema', 'PF'],   
            })
            
            # Muestra tabla resumida del backtesting
            print(f'\nRESUMEN VALIDACION:')
            print(f'   Operaciones: {dfBacktesting.loc["Sistema", "op"]:.0f}')
            print(f'   PA: {dfBacktesting.loc["Sistema", "pa%"]:.2f}%')
            print(f'   CAGR: {dfBacktesting.loc["Sistema", "cagr%"]:.2f}%')
            print(f'   Max DD: {dfBacktesting.loc["Sistema", "medDD%"]:.2f}%')
            print(f'   Profit Factor: {dfBacktesting.loc["Sistema", "PF"]:.2f}')  
            print(f'   {metricaSeleccion}: {metricaVal:.2f}') 
            print(f'   OCP: {dfBacktesting.loc["Sistema", "OCP"]:.2f}')
            
        except Exception as e:
            print(f'ERROR EN VALIDACION: {e}')
    else:
        print(f'No se pudo seleccionar ninguna variacion valida')

# ANALISIS FINAL DE RESULTADOS
print('=============================================================')
print(f'RESUMEN FINAL WALK FORWARD - METRICA: {metricaSeleccion}')
print('==============================================================')

if resultadosWF:
    resultadosDF = pd.DataFrame(resultadosWF)
    
    print(f'\nESTADISTICAS GENERALES:')
    print(f'   Operaciones promedio: {resultadosDF["op"].mean():.1f}')
    print(f'   PA promedio: {resultadosDF["Pa"].mean():.2f}%')
    print(f'   CAGR promedio: {resultadosDF["CAGR"].mean():.2f}%')
    print(f'   Max DD promedio: {resultadosDF["medDD"].mean():.2f}%')
    print(f'   PF promedio: {resultadosDF["PF"].mean():.2f}')
    print(f'   {metricaSeleccion} promedio en training: {resultadosDF["metricaTraining"].mean():.2f}')
    print(f'   {metricaSeleccion} promedio en validacion: {resultadosDF["metricaVal"].mean():.2f}')  
    print(f'   OCP promedio en validacion: {resultadosDF["OCP"].mean():.2f}')

    print(f'\nDETALLE POR BLOQUE:')
    for _, row in resultadosDF.iterrows():
        print(f'   Bloque {row["bloque"]}: Variacion {row["variacionElegida"]} → {metricaSeleccion}: {row["metricaTraining"]:.2f} | {metricaSeleccion} Val: {row["metricaVal"]:.2f} | OCP: {row["OCP"]:.2f} | PA: {row["Pa"]:.1f}%')  # CORREGIDO
    
    print('\nTABLA COMPLETA:')
    display(resultadosDF)
    
    # COMPONER CURVAS COMPLETAS
    print('\n' + '='*60)
    print('COMPONIENDO CURVAS DE VALIDACION')
    print('='*60)
    
    # Crear DataFrame con todas las fechas de validacion
    todasFechasVal = []
    for curva in curvasValidacion:
        todasFechasVal.extend(curva['fechas'])
    
    todasFechasVal = pd.Index(sorted(set(todasFechasVal)))
    
    curvaActivoCompleta = pd.Series(index=todasFechasVal, dtype=float)
    curvaSistemaCompleta = pd.Series(index=todasFechasVal, dtype=float)
    
    # Rellenar curvas
    valorFinalActivo = 100
    valorFinalSistema = 100
    
    for i, curva in enumerate(curvasValidacion):
        fechasBloque = curva['fechas']
        
        if i == 0:
            # Primer bloque - establecer valores iniciales
            curvaActivoCompleta.loc[fechasBloque] = curva['cvAct']
            curvaSistemaCompleta.loc[fechasBloque] = curva['cvSis']
            valorFinalActivo = curva['cvAct'].iloc[-1]
            valorFinalSistema = curva['cvSis'].iloc[-1]
        else:
            # Bloques siguientes - continuar desde el valor final anterior
            curvaActivoEscalada = (curva['cvAct'] / curva['cvAct'].iloc[0]) * valorFinalActivo
            curvaSistemaEscalada = (curva['cvSis'] / curva['cvSis'].iloc[0]) * valorFinalSistema
            
            curvaActivoCompleta.loc[fechasBloque] = curvaActivoEscalada
            curvaSistemaCompleta.loc[fechasBloque] = curvaSistemaEscalada
            
            valorFinalActivo = curvaActivoEscalada.iloc[-1]
            valorFinalSistema = curvaSistemaEscalada.iloc[-1]
    
    # GRAFICO FINAL - SOLO CURVAS
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(15, 8))
    
    plt.plot(curvaActivoCompleta.index, curvaActivoCompleta, 
             label='Activo', linewidth=3, color='black', alpha=0.8)
    plt.plot(curvaSistemaCompleta.index, curvaSistemaCompleta, 
             label=f'Sistema (Seleccion: {metricaSeleccion})', linewidth=2, color='red', alpha=0.8)
    
    plt.title(f'Walk Forward Validation - Seleccion por {metricaSeleccion}', fontsize=14, fontweight='bold')
    plt.ylabel('Valor de la Curva')
    plt.xlabel('Fecha')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f'\nRENDIMIENTO FINAL:')
    print(f'   Activo: {valorFinalActivo:.2f} ({((valorFinalActivo/100)-1)*100:.2f}%)')
    print(f'   Sistema: {valorFinalSistema:.2f} ({((valorFinalSistema/100)-1)*100:.2f}%)')
    
else:
    print('No se pudieron generar resultados')

#### (4) -> Cross Validation (ventana rolling)

NUNCA UTILIZAR DATOS FUTUROS PARA ENTRENAR UN BLOQUE

In [ ]:
perSma = 200 
perRsi = 4

rsiIn  = 40
rsiOut = 70

salidaVelas = 0

sentido = 'long'

pSize = 1

tp = 0
sl = 0

# 0.01 supone un $1 por cada $10k
# 0.03 como maximo, en diario es 0
comision = 0
slippage = 0

# [perSma, perRsi, rsiIn, rsiOut, salidaVelas, sentido, pSize tp, sl, comision, slippage]
variacionesSistema = [[200, 4, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 4, 10, 50, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 40, 70, 0, 'long',1, 0, 0, 0, 0],     
                      [200, 2, 10, 50, 0, 'long',1, 0, 0, 0, 0]
                     ]

In [ ]:
# Cambiar por: 'OCP', 'PF', 'cagr%', 'pa%', 'roi%', etc.
metricaSeleccion = 'PF'  

numBloques = 5

In [ ]:
datos = pd.read_excel('dfAll.xlsx', index_col=0)
print(f'Datos originales: {datos.index[0]} a {datos.index[-1]} ({len(datos)} días)')

datos = datos[datos.index.year >= 2017].copy()
print(f'Datos desde 2017: {datos.index[0]} a {datos.index[-1]} ({len(datos)} días)')

print(f'METRICA DE SELECCION: {metricaSeleccion}')
print('='*50)

# CROSS VALIDATION VENTANA FIJA (solo bloque anterior)
resultadosCV = []
curvasValidacion = []  # Para recoger las curvas de cada validacion
tamBloque = len(datos) // numBloques

for bloque in range(1, numBloques):  # Empieza en 1 porque el primer bloque no tiene anterior para entrenar
    
    print(f'\n=============================================')
    print(f'BLOQUE {bloque}/{numBloques-1}')
    print(f'=============================================')
    
    # Definir INDICES de ventanas
    inicioTrain = (bloque - 1) * tamBloque
    finTrain = bloque * tamBloque
    inicioVal = finTrain
    
    if bloque == numBloques - 1:
        finVal = len(datos)
    else:
        finVal = min(finTrain + tamBloque, len(datos))
    
    # Verificar que hay datos suficientes
    if inicioVal >= len(datos) or finVal - inicioVal < 50:
        print('No hay datos suficientes para validacion')
        continue
    
    print(f'TRAINING: {datos.index[inicioTrain]} a {datos.index[finTrain-1]} ({finTrain-inicioTrain} dias)')
    print(f'VALIDACION: {datos.index[inicioVal]} a {datos.index[finVal-1]} ({finVal-inicioVal} dias)')
    
    # OPTIMIZACION EN TRAINING
    print(f'\nEVALUANDO VARIACIONES EN TRAINING:')
    print('----------------------------------------')
    
    mejorVariacion = None
    mejorScore = -999
    mejorIndice = -1
    
    for i, variacion in enumerate(variacionesSistema):
        try:
            _, dfBacktestingTrain = evaluarSistema(datos, variacion, inicioTrain, finTrain)
            score = dfBacktestingTrain.loc['Sistema', metricaSeleccion]
            
            print(f'Variacion {i}: {variacion} → {metricaSeleccion}: {round(score,2)}')
            
            if score > mejorScore:
                mejorScore = score
                mejorVariacion = variacion
                mejorIndice = i
                
        except Exception as e:
            print(f'Variacion {i}: ERROR - {e}')
            continue
    
    print('-----------------------------------------------')
    print(f'MEJOR EN TRAINING: Variacion {mejorIndice} → {metricaSeleccion}: {mejorScore:.2f}')
    print(f'   Parametros: {mejorVariacion}')
    
    # APLICAR MEJOR SISTEMA A VALIDACION
    if mejorVariacion is not None:
        try:
            print(f'\nAPLICANDO VARIACION {mejorIndice} EN VALIDACION...')
            scoreVal, dfBacktesting = evaluarSistema(datos, mejorVariacion, inicioVal, finVal)
            
            # Mostrar la métrica de validación correspondiente
            metricaVal = dfBacktesting.loc['Sistema', metricaSeleccion]
            print(f'RESULTADO VALIDACION: {metricaSeleccion} {metricaVal:.2f} | OCP {scoreVal:.2f}')
            
            # OBTENER CURVAS APLICANDO EL SISTEMA COMPLETO A LA VALIDACION
            perSma, perRsi, rsiIn, rsiOut, salidaVelas, sentido, pSize, tp, sl, comision, slippage = mejorVariacion
            
            dataValidacion = datos.copy()
            dataValidacion = ocpSma(dataValidacion, perSma)
            dataValidacion = ocpRsi(dataValidacion, perRsi, True)
            dataValidacion = dameSistema(dataValidacion, perSma, perRsi, rsiIn, rsiOut)
            dataValidacion = damePosition(dataValidacion)
            dataValidacion = dameSalidaVelas(dataValidacion, salidaVelas)
            dataValidacion = dameSalidaPnl(dataValidacion, sentido, tp, sl, comision, slippage)
            dataValidacion = dataValidacion.iloc[inicioVal:finVal].copy()
            dataValidacion = calculaCurvas(dataValidacion, pSize)
            
            # RECOGER CURVAS DE VALIDACION
            curvasValidacion.append({
                'bloque': bloque,
                'fechas': dataValidacion.index,
                'cvAct': dataValidacion['cvAct'],
                'cvSis': dataValidacion['cvSis'],
                'inicioVal': inicioVal,
                'finVal': finVal
            })
            
            resultadosCV.append({
                'bloque': bloque,
                'training': f'{datos.index[inicioTrain].strftime("%Y-%m")} a {datos.index[finTrain-1].strftime("%Y-%m")}',
                'metricaTraining': mejorScore,
                'variacionElegida': mejorIndice,
                'parametros': mejorVariacion,
                'validation': f'{datos.index[inicioVal].strftime("%Y-%m")} a {datos.index[finVal-1].strftime("%Y-%m")}',
                'metricaVal': metricaVal,  
                'OCP': scoreVal,
                'op': dfBacktesting.loc['Sistema', 'op'],
                'Pa': dfBacktesting.loc['Sistema', 'pa%'],
                'CAGR': dfBacktesting.loc['Sistema', 'cagr%'],
                'medDD': dfBacktesting.loc['Sistema', 'medDD%'],
                'PF': dfBacktesting.loc['Sistema', 'PF'],   
            })
            
            # Mostrar tabla resumida del backtesting
            print(f'\nRESUMEN VALIDACION:')
            print(f'   Operaciones: {dfBacktesting.loc["Sistema", "op"]:.0f}')
            print(f'   PA: {dfBacktesting.loc["Sistema", "pa%"]:.2f}%')
            print(f'   CAGR: {dfBacktesting.loc["Sistema", "cagr%"]:.2f}%')
            print(f'   Max DD: {dfBacktesting.loc["Sistema", "medDD%"]:.2f}%')
            print(f'   Profit Factor: {dfBacktesting.loc["Sistema", "PF"]:.2f}')  
            print(f'   {metricaSeleccion}: {metricaVal:.2f}')  # CORREGIDO
            print(f'   OCP: {dfBacktesting.loc["Sistema", "OCP"]:.2f}')
            
        except Exception as e:
            print(f'ERROR EN VALIDACION: {e}')
    else:
        print(f'No se pudo seleccionar ninguna variacion valida')

# ANALISIS FINAL DE RESULTADOS
print('=============================================================')
print(f'RESUMEN FINAL CROSS VALIDATION - METRICA: {metricaSeleccion}')
print('==============================================================')

if resultadosCV:
    resultadosDF = pd.DataFrame(resultadosCV)
    
    print(f'\nESTADISTICAS GENERALES:')
    print(f'   Operaciones promedio: {resultadosDF["op"].mean():.1f}')
    print(f'   PA promedio: {resultadosDF["Pa"].mean():.2f}%')
    print(f'   CAGR promedio: {resultadosDF["CAGR"].mean():.2f}%')
    print(f'   Max DD promedio: {resultadosDF["medDD"].mean():.2f}%')
    print(f'   PF promedio: {resultadosDF["PF"].mean():.2f}')
    print(f'   {metricaSeleccion} promedio en training: {resultadosDF["metricaTraining"].mean():.2f}')
    print(f'   {metricaSeleccion} promedio en validacion: {resultadosDF["metricaVal"].mean():.2f}') 
    print(f'   OCP promedio en validacion: {resultadosDF["OCP"].mean():.2f}')

    print(f'\nDETALLE POR BLOQUE:')
    for _, row in resultadosDF.iterrows():
        print(f'   Bloque {row["bloque"]}: Variacion {row["variacionElegida"]} → {metricaSeleccion}: {row["metricaTraining"]:.2f} | {metricaSeleccion} Val: {row["metricaVal"]:.2f} | OCP: {row["OCP"]:.2f} | PA: {row["Pa"]:.1f}%')  # CORREGIDO
    
    print('\nTABLA COMPLETA:')
    display(resultadosDF)
    
    # COMPONER CURVAS COMPLETAS
    print('\n' + '='*60)
    print('COMPONIENDO CURVAS DE VALIDACION')
    print('='*60)
    
    # Crear DataFrame con todas las fechas de validacion
    todasFechasVal = []
    for curva in curvasValidacion:
        todasFechasVal.extend(curva['fechas'])
    
    todasFechasVal = pd.Index(sorted(set(todasFechasVal)))
    
    curvaActivoCompleta = pd.Series(index=todasFechasVal, dtype=float)
    curvaSistemaCompleta = pd.Series(index=todasFechasVal, dtype=float)
    
    # Rellenar curvas
    valorFinalActivo = 100
    valorFinalSistema = 100
    
    for i, curva in enumerate(curvasValidacion):
        fechasBloque = curva['fechas']
        
        if i == 0:
            # Primer bloque - establecer valores iniciales
            curvaActivoCompleta.loc[fechasBloque] = curva['cvAct']
            curvaSistemaCompleta.loc[fechasBloque] = curva['cvSis']
            valorFinalActivo = curva['cvAct'].iloc[-1]
            valorFinalSistema = curva['cvSis'].iloc[-1]
        else:
            # Bloques siguientes - continuar desde el valor final anterior
            curvaActivoEscalada = (curva['cvAct'] / curva['cvAct'].iloc[0]) * valorFinalActivo
            curvaSistemaEscalada = (curva['cvSis'] / curva['cvSis'].iloc[0]) * valorFinalSistema
            
            curvaActivoCompleta.loc[fechasBloque] = curvaActivoEscalada
            curvaSistemaCompleta.loc[fechasBloque] = curvaSistemaEscalada
            
            valorFinalActivo = curvaActivoEscalada.iloc[-1]
            valorFinalSistema = curvaSistemaEscalada.iloc[-1]
    
    # GRAFICO FINAL - SOLO CURVAS  
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(15, 8))
    
    plt.plot(curvaActivoCompleta.index, curvaActivoCompleta, 
             label='Activo', linewidth=3, color='black', alpha=0.8)
    plt.plot(curvaSistemaCompleta.index, curvaSistemaCompleta, 
             label=f'Sistema (Seleccion: {metricaSeleccion})', linewidth=2, color='red', alpha=0.8)
    
    plt.title(f'Cross Validation - Seleccion por {metricaSeleccion}', fontsize=14, fontweight='bold')
    plt.ylabel('Valor de la Curva')
    plt.xlabel('Fecha')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f'\nRENDIMIENTO FINAL:')
    print(f'   Activo: {valorFinalActivo:.2f} ({((valorFinalActivo/100)-1)*100:.2f}%)')
    print(f'   Sistema: {valorFinalSistema:.2f} ({((valorFinalSistema/100)-1)*100:.2f}%)')
    
else:
    print('No se pudieron generar resultados')

### VALIDACION MKT

#### (1) -> Prueba en paper

#### (2) -> Prueba con poco capital